# Tips
## Deep copy
### Problem definition
Both lists and dicts have methods for creating copies of these objects:

Creating a copy of a list:

In [3]:
list1 = [1, 2, 3]
list2 = list1[:]

list2 is list1

False

Copy of a dict:

In [4]:
dict1 = dict(a=1, b=2, c=3)
dict2 = dict1.copy()

dict2 is dict1

False

However this could be a problem in case your list (or dict) is not just a list, but a complex structure. Both `copy()`s actually copy only the referenced object contents. And because in a complex structure, elements store references to other complex (and usually mutable) objects, the copied object store them as well. This can lead to an undesired behaviour that we usually call **bugs**. Example demonstrating the problem:

In [7]:
crew = [
    dict(firstname='Tomas', lastname='Anderson'),
    dict(firstname='Morpheus'),
    dict(firstname='Trinity'),
    dict(firstname='Cypher'),
]

because `database` is a list, we use `[:]` to get a copy of it. 

In [9]:
replica = crew[:]

Now we may be thinking that `replica` is a full copy of `crew`, we use it to address and modify values:

In [10]:
replica[1]['firstname']

'Morpheus'

We can safely delete or replace items in `replica`.

In [12]:
replica[2] = dict(firstname='Mouse')
del replica[3]

Look, original `crew` is kept untouched:

In [13]:
crew

[{'firstname': 'Tomas', 'lastname': 'Anderson'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]

However if we attempt to modify a deeper value, like so:

In [16]:
replica[0]['firstname'] = 'Neo'
replica[0]['lastname'] = 'The One'

`crew` appears to be modified, which is probably undesired.

In [17]:
crew

[{'firstname': 'Neo', 'lastname': 'The One'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]

### The better way: deepcopy

In [19]:
from copy import deepcopy

Defining `crew` once again:

In [23]:
crew = [
    dict(firstname='Tomas', lastname='Anderson'),
    dict(firstname='Morpheus'),
    dict(firstname='Trinity'),
    dict(firstname='Cypher'),
]

In [24]:
replica = deepcopy(crew)

In [25]:
replica[0]['firstname'] = 'Neo'
replica[0]['lastname'] = 'The One'

replica

[{'firstname': 'Neo', 'lastname': 'The One'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]

`crew` remains unmodified:

In [26]:
crew

[{'firstname': 'Tomas', 'lastname': 'Anderson'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]